In [1]:
### GEOCODE DATAFRAME WITH GEOPY
# cf : https://geopy.readthedocs.io/en/stable/#installation
# cf : https://wiki.openstreetmap.org/wiki/Nominatim
# cf : http://blog.adrienvh.fr/2015/01/18/geocoder-en-masse-plusieurs-milliers-dadresses-avec-python-et-nominatim/

from time import sleep

import pprint
pp = pprint.PrettyPrinter(indent=2)

import pandas as pd

# cf : https://github.com/jmcarpenter2/swifter
# cf : https://medium.com/@jmcarpenter2/swiftapply-automatically-efficient-pandas-apply-operations-50e1058909f9
import swifter

from geopy.geocoders import Nominatim, BANFrance
from geopy.extra.rate_limiter import RateLimiter

from functools import partial
from tqdm import tqdm

In [2]:
tqdm.pandas()

**GENERIC VARIABLES**

In [32]:
dft_delay        = 1.5
dft_timeout      = 20
exact_one_dft    = True
full_address_col = "_full_address_"
location_col     = "_location_"

**GENERIC FUNCTIONS**

In [4]:
geocoder_nom = Nominatim(user_agent="_solidata_app_")
geocoder_ban = BANFrance(user_agent="_solidata_app_")

In [5]:
### rate limiter
geocode_nom = RateLimiter(geocoder_nom.geocode, min_delay_seconds=dft_delay)
geocode_ban = RateLimiter(geocoder_ban.geocode, min_delay_seconds=dft_delay)

In [6]:
def LocToDict(location) : 
    if location != None : 
        return {
            "raw"       : location.raw,
            "address"   : location.address,
            "point"     : location.point,
            "latitude"  : location.latitude,
            "longitude" : location.longitude,
        }
    else : 
        return {
            "raw"        : None,
            "address"    : None,
            "point"      : None,
            "latitude"   : None,
            "longitude"  : None,
        }

In [7]:
### concat function
def concat_cols(row, columns_to_concat):
    if len(columns_to_concat) > 1 :
        return ", ".join( row[col] for col in columns_to_concat )
    else : 
        return row[columns_to_concat[0]]

In [8]:
### main geolocalizing function for dataframe
def geoloc_df( row, col_name, complement, time_out, delay ) : 

    print ("\n- row[col_name] : ", row[col_name])

    adress = ", ".join( [row[col_name], complement ] )
    print ("- adress : ", adress)
    
    try :
        location = geocoder_nom.geocode( query=adress, timeout=time_out, extratags=True)
    except : 
        location = geocoder_ban.geocode( query=adress, timeout=time_out)
        
    print ("- location : ", location)
    
    sleep(delay)
    
    if location : 
        return LocToDict(location)
    else : 
        return None
    

In [33]:
### main geolocalizing function for dataframe
def geoloc_df_col( 
    row_val, 
    complement="", 
    time_out=dft_timeout, 
    delay=dft_delay,
    exact_one=exact_one_dft
    ) : 

    print ("\n- row_val : ", row_val)
    
    if pd.notnull(row_val) : 
        
        adress = ", ".join( [ row_val, complement ] )
        print ("- adress : ", adress)

        try :
            location = geocoder_nom.geocode( query=adress, exactly_one=exact_one, timeout=time_out, extratags=True)
        except : 
            location = geocoder_ban.geocode( query=adress, exactly_one=exact_one, timeout=time_out)

        print ("- location : ", location)

        sleep(delay)

        if location : 
            return LocToDict(location)
        
        else : 
            return None
    
    else : 
        return None

**test with simple address**

In [10]:
### variables to geocode

adresse_01 = "191 rue Saint-Jacques, Paris, France"
adresse_02 = "ABERGEMENT CLEMENCIAT, 01001"

adress_complement = "France"

In [11]:
location_01 = geocoder_nom.geocode(query=adresse_01, timeout=dft_timeout, extratags=True)
print(location_01)

Institut de Géographie, 191, Rue Saint-Jacques, Val-de-Grâce, 5e, Paris, Île-de-France, France métropolitaine, 75005, France


In [12]:
location_01_dict = LocToDict(location_01)
pp.pprint(location_01_dict)

{ 'address': 'Institut de Géographie, 191, Rue Saint-Jacques, Val-de-Grâce, '
             '5e, Paris, Île-de-France, France métropolitaine, 75005, France',
  'latitude': 48.84468875,
  'longitude': 2.34258870005006,
  'point': Point(48.84468875, 2.34258870005006, 0.0),
  'raw': { 'boundingbox': [ '48.8445388',
                            '48.8448244',
                            '2.3423236',
                            '2.3428539'],
           'class': 'amenity',
           'display_name': 'Institut de Géographie, 191, Rue Saint-Jacques, '
                           'Val-de-Grâce, 5e, Paris, Île-de-France, France '
                           'métropolitaine, 75005, France',
           'extratags': { 'internet_access': 'no',
                          'wikidata': 'Q3152006',
                          'wikipedia': 'fr:Institut de géographie'},
           'icon': 'https://nominatim.openstreetmap.org/images/mapicons/education_university.p.20.png',
           'importance': 0.827646119222430

In [13]:
location_02a = geocoder_nom.geocode(adresse_02, True, dft_timeout)
print(location_02a)

None


In [14]:
location_02b = geocoder_ban.geocode(adresse_02, timeout=dft_timeout)
print((location_02b.latitude, location_02b.longitude))

(46.150598, 4.920347)


In [15]:
location_02_dict = LocToDict(location_02b)
pp.pprint(location_02_dict)

{ 'address': "L'Abergement-Clémenciat",
  'latitude': 46.150598,
  'longitude': 4.920347,
  'point': Point(46.150598, 4.920347, 0.0),
  'raw': { 'geometry': {'coordinates': [4.920347, 46.150598], 'type': 'Point'},
           'properties': { 'adm_weight': 1,
                           'city': "L'Abergement-Clémenciat",
                           'citycode': '01001',
                           'context': '01, Ain, Auvergne-Rhône-Alpes '
                                      '(Rhône-Alpes)',
                           'id': '01001',
                           'importance': 0.075,
                           'label': "L'Abergement-Clémenciat",
                           'name': "L'Abergement-Clémenciat",
                           'population': 0.8,
                           'postcode': '01400',
                           'score': 0.6613636363636364,
                           'type': 'municipality',
                           'x': 848200,
                           'y': 6563000},
        

**test with pandas**

In [16]:
csv_test_01 = "tests_jupyter/2016_codes_postaux_prep.csv"
csv_test_02 = "tests_jupyter/export_openscraper_5aa3de630a828651b2ebfe3e.csv"

In [17]:
df_01 = pd.read_csv(csv_test_01, encoding="utf8")
df_01.head(3)

,Code_commune_INSEE,Nom_commune,Code_postal,Libelle_acheminement,Ligne_5,latitude,longitude,Population,Surface,geopoint_2d
0,01001,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,NaN,46.156818,4.924707,767.0,1567.0,POINT(4.924707496412453 46.15681799286022)
1,01002,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,NaN,46.010085,5.428759,236.0,909.0,POINT(5.428759087785754 46.01008524345922)
2,01004,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,45.958400,5.375901,14359.0,2449.0,POINT(5.375901233977878 45.958399862151865)


In [18]:
df_02 = pd.read_csv(csv_test_02, sep="|", encoding="utf8")
df_02.head(3)

,spider_name,link_src,link_data,fiche contact,date du projet,partenaires du projet,données économiques,résumé du projet,tags,website,auteur,logo,adresse du projet,titre du projet,image(s) du projet,structure porteuse,SIRET,statut juridique,récompenses,video
0,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,82 avenue Jean Jaurès Directrice du pole Mail:...,• APAJH • Association Charles de Foucauld Cons...,"Afin d’améliorer l’accueil, de plus en plus fr...","Gouvernance, partenariats institutionnels Vie ...",mailto:gwenola.toutain@croix-rouge.fr,Bourgogne,Accompagnement à la parentalité : des acteurs ...,http://www.apriles.net/images/stories/Centrepa...,Centre parental de Migennes - Croix Rouge fran...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: Mail: 72 Rue Président Wilson 4...,- Proposer des fruits et légumes frais de sais...,«Relocaliser la production agricole a des impa...,"Gouvernance, partenariats institutionnels Jeun...",http://www.grandcahors.fr mailto:dpennequin@gr...,Midi-Pyrénées,La légumerie : première pierre du Projet alime...,http://www.apriles.net/images/stories/Le%CC%81...,Communauté d'agglomération du Grand Cahors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,"Lien internet: 44100 Nantes Directrice, Pôle é...","Etat (aides à la pierre), Région Pays-de-la-Lo...",Afin de répondre aux besoins d’hébergement de ...,"Petite enfance, modes de garde Parentalité Rel...",mailto:Catherine.bouget@croix-rouge.fr http://...,Pays de la Loire,"Au Bois Bouchaud, parents, enfants et seniors ...",http://www.apriles.net/images/stories/Boisbouc...,Croix Rouge française,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_01.columns

Index(['Code_commune_INSEE', 'Nom_commune', 'Code_postal',
       'Libelle_acheminement', 'Ligne_5', 'latitude', 'longitude',
       'Population', 'Surface', 'geopoint_2d'],
      dtype='object')

In [20]:
df_02.columns

Index(['spider_name', 'link_src', 'link_data', 'fiche contact',
       'date du projet', 'partenaires du projet', 'données économiques',
       'résumé du projet', 'tags', 'website', 'auteur', 'logo',
       'adresse du projet', 'titre du projet', 'image(s) du projet',
       'structure porteuse', 'SIRET', 'statut juridique', 'récompenses',
       'video'],
      dtype='object')

In [21]:
### list of meaningfull columns for geoloc
address_columns_01 = ["Nom_commune", "Code_commune_INSEE"]
address_columns_02 = ["adresse du projet"]

In [22]:
addresses = df_01["Nom_commune"].tolist()
print (addresses[:5], "...")

['L ABERGEMENT CLEMENCIAT', 'L ABERGEMENT DE VAREY', 'AMBERIEU EN BUGEY', 'AMBERIEUX EN DOMBES', 'AMBLEON'] ...


In [23]:
### apply concat function to each row (axis=1)
df_01[full_address_col] = df_01.apply(concat_cols, args=[address_columns_01], axis=1)
df_01.head(3)

,Code_commune_INSEE,Nom_commune,Code_postal,Libelle_acheminement,Ligne_5,latitude,longitude,Population,Surface,geopoint_2d,_full_address_
0,01001,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,NaN,46.156818,4.924707,767.0,1567.0,POINT(4.924707496412453 46.15681799286022),"L ABERGEMENT CLEMENCIAT, 01001"
1,01002,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,NaN,46.010085,5.428759,236.0,909.0,POINT(5.428759087785754 46.01008524345922),"L ABERGEMENT DE VAREY, 01002"
2,01004,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,45.958400,5.375901,14359.0,2449.0,POINT(5.375901233977878 45.958399862151865),"AMBERIEU EN BUGEY, 01004"


In [24]:
df_01.shape

(39201, 11)

In [25]:
### apply concat function to each row (axis=1)
df_02[full_address_col] = df_02.apply(concat_cols, args=[address_columns_02], axis=1)
df_02.sample(3)

,spider_name,link_src,link_data,fiche contact,date du projet,partenaires du projet,données économiques,résumé du projet,tags,website,...,logo,adresse du projet,titre du projet,image(s) du projet,structure porteuse,SIRET,statut juridique,récompenses,video,_full_address_
248,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,La Protection Judiciaire de la Jeunesse Le Con...,Afin d'offrir une alternative au placement des...,Protection de l'enfance Parentalité Liens inte...,Rhône-Alpes,"En Haute-Savoie, Retis-ser des liens pour mieu...","Association Retis (Recherche, éducation, terri...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: Adjoint au maire Ville de Mouan...,Pas de partenariat pour la régie agricole.,Financiers : Un domaine d’une surface utile de...,Afin de promouvoir une alimentation saine et é...,"Emploi, Formation Développement local rural Go...",mailto:gilles.perole@mouans-sartoux.net http:/...,Provence-Alpes-Côte d'Azur,...,http://www.apriles.net/images/stories/Mouans-S...,Ville de Mouans-Sartoux,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville de Mouans-Sartoux
42,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: Mail: 59500 Douai Tel: 03 27 95...,NaN,Afin d’améliorer les conditions de vie des loc...,Rénovation urbaine Participation des habitants...,http://www.soliha.fr/ mailto:v.simon@soliha.fr,Nord-Pas-de-Calais,Mon Eco Logement : quand la rénovation donne d...,...,Soliha Douaisis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df_02.shape

(269, 21)

In [27]:
### MAIN GEOCODING FUNCTION ON DF + SWIFTER

df_02[location_col] = df_02[full_address_col].swifter.apply( 
    geoloc_df_col, 
    complement=adress_complement, 
    #time_out=dft_timeout, 
    #delay=dft_delay, 
)


- row_val :  0                                                   NaN
1                                                   NaN
2                                                   NaN
3                                                   NaN
4                            Centre hospitalier du Mans
5                            Coopérer pour entreprendre
6                               Ville de Mouans-Sartoux
7     Centre de Ressources Politique de la Ville de ...
8                                     Ville de Bordeaux
9                                    ENVIE Autonomie 49
10                                  Association Viltaïs
11                            Ville de Clichy-sous-Bois
12                                        Ville d’Arras
13                                                  NaN
14                                        CCAS d’Angers
15                                                  NaN
Name: _full_address_, dtype: object

- row_val :  nan

- row_val :  nan

- row_val :  nan



- row_val :  Coopérer pour entreprendre
- adress :  Coopérer pour entreprendre, France
- location :  None

- row_val :  Ville de Mouans-Sartoux
- adress :  Ville de Mouans-Sartoux, France
- location :  None

- row_val :  Centre de Ressources Politique de la Ville de Guyane (Commissariat général à l’égalité des territoires – CGET)
- adress :  Centre de Ressources Politique de la Ville de Guyane (Commissariat général à l’égalité des territoires – CGET), France
- location :  None

- row_val :  Ville de Bordeaux
- adress :  Ville de Bordeaux, France
- location :  Base sous-marine de Bordeaux, Boulevard Alfred Daney, Bordeaux Maritime, Bordeaux, Gironde, Nouvelle-Aquitaine, France métropolitaine, 33000, France

- row_val :  ENVIE Autonomie 49
- adress :  ENVIE Autonomie 49, France
- location :  None

- row_val :  Association Viltaïs
- adress :  Association Viltaïs, France
- location :  None

- row_val :  Ville de Clichy-sous-Bois
- adress :  Ville de Clichy-sous-Bois, France
- location :  

- location :  Centre Hospitalier du Mans, Rue de Degré, Cité de l'Épine, Le Buisson, Le Mans, Sarthe, Pays de la Loire, France métropolitaine, 72000, France

- row_val :  Coopérer pour entreprendre
- adress :  Coopérer pour entreprendre, France
- location :  None

- row_val :  Ville de Mouans-Sartoux
- adress :  Ville de Mouans-Sartoux, France
- location :  None

- row_val :  Centre de Ressources Politique de la Ville de Guyane (Commissariat général à l’égalité des territoires – CGET)
- adress :  Centre de Ressources Politique de la Ville de Guyane (Commissariat général à l’égalité des territoires – CGET), France
- location :  None

- row_val :  Ville de Bordeaux
- adress :  Ville de Bordeaux, France
- location :  Base sous-marine de Bordeaux, Boulevard Alfred Daney, Bordeaux Maritime, Bordeaux, Gironde, Nouvelle-Aquitaine, France métropolitaine, 33000, France

- row_val :  ENVIE Autonomie 49
- adress :  ENVIE Autonomie 49, France
- location :  None

- row_val :  Association Viltaïs
- 

Pandas Apply:   0%|          | 0/269 [00:00<?, ?it/s]


- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val

...


- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  Association Energies et jardins du c'ur
- adress :  Association Energies et jardins du c'ur, France
- location :  None


Pandas Apply:  56%|█████▌    | 151/269 [00:42<00:49,  2.40it/s]


- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  Association la Source
- adress :  Association la Source, France
- location :  Association Pêcheurs Libres, Quai des Anglais, La Source, Saint-Pierre, Martigues, Istres, Bouches-du-Rhône, Provence-Alpes-Côte d'Azur, France métropolitaine, 13500, France


Pandas Apply: 100%|██████████| 269/269 [00:44<00:00,  6.11it/s]


- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val

In [28]:
df_02.sample(10)

,spider_name,link_src,link_data,fiche contact,date du projet,partenaires du projet,données économiques,résumé du projet,tags,website,...,adresse du projet,titre du projet,image(s) du projet,structure porteuse,SIRET,statut juridique,récompenses,video,_full_address_,_location_
101,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Tel: 01 34 30 00 30 responsable du pôle réseau...,"Association école et famille, politique de la ...",Au collège Montaigne de Goussainville (Val-d’O...,Diagnostic partagé Participation des habitants...,http://www.ecoleetfamille.fr/ mailto:pole.rese...,Île-de-France,"A Goussainville, le collège fait tomber les murs",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Coordinatrice ULIS Collège Marie Mauron 84120 ...,NaN,Afin de favoriser l’inclusion de jeunes collég...,"Accessibilité, Mobilité Intégration Bénévolat ...",mailto:fraisse.ulis@hotmail.fr,Provence-Alpes-Côte d'Azur,Handicap : quand EVA inclut des collégiens au ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: 10 rue de la Cassotte Mail: Tel...,"Au niveau du réseau : Ashoka, ESSEC'Fédération...","Face à l'augmentation de la discrimination, de...","Vie en établissement Logement, Habitat Lutte c...",mailto:mnb@lesinvitesaufestin.fr http://www.le...,Franche-Comt,Une prise en charge citoyenne des troubles psy...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Direction générale adjointe de l’Animation et ...,"Université Lumière Lyon 2, Maison de Quartier ...",Pour en savoir plus : ο « Les groupes centrés ...,Afin d’améliorer l’action publique menée sur d...,Nouvelles pratiques professionnelles Diagnosti...,http://www.villeurbanne.fr/ mailto:simon.mover...,Rhône-Alpes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Mail: 26270 Loriol-sur-Drôme Responsable du ce...,-Le Conseil général de la Drôme -L'Education n...,Afin de réduire les actes d'incivilité sur la ...,Relations de voisinage Lutte contre l'exclusio...,mailto:espacejeune@loriol.com,Rhône-Alpes,Des chantiers contre les murs de l'isolement - 26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: Animateur de la démarche ( se d...,"Conseil départemental de la Drôme, Pôle Emploi...",En partenariat avec le Conseil départemental d...,Accès aux droits Participation des habitants L...,http://www.aequitaz.org/aventurescollectives/b...,Rhône-Alpes,"Une boussole, des cartes et les étoiles : un n...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: Laboratoire Limousin de la laïc...,Caf de Haute Vienne Kiwanis International Fond...,Afin de promouvoir la laïcité et favoriser le ...,"Développement urbain, Vie des quartiers Relati...",http://vivreensembledanslacite.org mailto:phil...,Limousin,« Vivre ensemble dans la cité » : les habitant...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Régime social des indépendants(RSI) DGS CNSA,Pour répondre de manière continue aux besoins ...,"Gouvernance, partenariats institutionnels Nouv...",-France entière,"PRISMA, un modèle de prise en charge intégrée ...",Hôpital européen Georges Pompidou,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php

In [29]:
filtered_df = df_02[df_02[location_col].notnull()].copy()
filtered_df.sample(5)

,spider_name,link_src,link_data,fiche contact,date du projet,partenaires du projet,données économiques,résumé du projet,tags,website,...,adresse du projet,titre du projet,image(s) du projet,structure porteuse,SIRET,statut juridique,récompenses,video,_full_address_,_location_
4,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: Mail: 194 Avenue Rubillard Cent...,"Société Mécapack, don de barquettes à usage un...",Le fonctionnement actuel mobilise uniquement d...,Pour lutter contre le gaspillage et favoriser ...,Vie en établissement Intégration Lutte contre ...,http://www.ch-lemans.fr mailto:juteau@free.fr,Pays de la Loire,...,Centre hospitalier du Mans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Centre hospitalier du Mans,"{'raw': {'place_id': '109869850', 'licence': '..."
8,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: chargée de mission handicap et ...,"Préfecture, Direction régionale et département...",• Un chargé de mission analyse des besoins soc...,Afin d’améliorer la coordination et l’interpré...,"Gouvernance, partenariats institutionnels Diag...",http://www.bordeaux.fr mailto:c.beaufort-lance...,Aquitaine,...,Ville de Bordeaux,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville de Bordeaux,"{'raw': {'place_id': '82004620', 'licence': 'D..."
156,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Association la Source Lien internet: Mail: 3 r...,"En novembre 1994, la SCI des Amis de la Poulti...",___________________,>>Consulter la brochure des 20 ans de La Sourc...,Parentalité Lutte contre l'exclusion sociale P...,http://www.associationlasource.fr mailto:rober...,Haute-Normandie,...,Association la Source,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Association la Source,"{'raw': {'place_id': '63721638', 'licence': 'D..."
12,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,"Lien internet: Mairie d'Arras, Place Guy Molle...",Les équipes pluridisciplinaires de quartier ra...,1. Le Projet de Développement Solidaire d’Arra...,Afin de renforcer l’interconnaissance et la co...,"Gouvernance, partenariats institutionnels Déve...",http://www.arras.fr mailto:a-konieczny@ville-a...,Nord-Pas-de-Calais,...,Ville d’Arras,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville d’Arras,"{'raw': {'place_id': '61791374', 'licence': 'D..."
17,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Fabrique du quartier Danton – Ville du Havre L...,Sur la concertation menée sur le quartier Dant...,700 000 euros investis spécifiquement sur la c...,Afin de mettre les habitants au cœur du projet...,"Rénovation urbaine Développement urbain, Vie d...",mailto:benjamin.lebourg@lehavre.fr http://www....,Haute-Normandie,...,Ville du Havre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville du Havre,"{'raw': {'place_id': '22957963', 'licence': 'D..."


In [31]:
def extract_loc(row, field_name="longitude") :
    # print(row)
    # print (type(row))
    return row[field_name]
    
filtered_df["raw"] = filtered_df[location_col].apply(extract_loc, field_name="raw")
filtered_df["lat"] = filtered_df[location_col].apply(extract_loc, field_name="latitude")
filtered_df["long"] = filtered_df[location_col].apply(extract_loc, field_name="longitude")
filtered_df["address"] = filtered_df[location_col].apply(extract_loc, field_name="address")
filtered_df.sample(5)

,spider_name,link_src,link_data,fiche contact,date du projet,partenaires du projet,données économiques,résumé du projet,tags,website,...,SIRET,statut juridique,récompenses,video,_full_address_,_location_,raw,lat,long,address
17,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Fabrique du quartier Danton – Ville du Havre L...,Sur la concertation menée sur le quartier Dant...,700 000 euros investis spécifiquement sur la c...,Afin de mettre les habitants au cœur du projet...,"Rénovation urbaine Développement urbain, Vie d...",mailto:benjamin.lebourg@lehavre.fr http://www....,Haute-Normandie,...,NaN,NaN,NaN,NaN,Ville du Havre,"{'raw': {'place_id': '22957963', 'licence': 'D...","{'place_id': '22957963', 'licence': 'Data © Op...",49.512893,0.176707,"Médiathèque de Caucriauville, 40, Rue Jules Va..."
156,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Association la Source Lien internet: Mail: 3 r...,"En novembre 1994, la SCI des Amis de la Poulti...",___________________,>>Consulter la brochure des 20 ans de La Sourc...,Parentalité Lutte contre l'exclusion sociale P...,http://www.associationlasource.fr mailto:rober...,Haute-Normandie,...,NaN,NaN,NaN,NaN,Association la Source,"{'raw': {'place_id': '63721638', 'licence': 'D...","{'place_id': '63721638', 'licence': 'Data © Op...",43.404823,5.055001,"Association Pêcheurs Libres, Quai des Anglais,..."
19,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: 3 Quai Ceineray Tel: 02 40 99 1...,Le dispositif est entièrement internalisé. Il ...,Financiers : Quatre travailleurs sociaux du Dé...,Afin d’apporter une solution d’accueil adaptée...,Protection de l'enfance Intégration Nouvelles ...,mailto:laurent.gollandeau@loire-atlantique.fr ...,Pays de la Loire,...,NaN,NaN,NaN,NaN,Conseil départemental de Loire-Atlantique,"{'raw': {'place_id': '241565046', 'licence': '...","{'place_id': '241565046', 'licence': 'Data © O...",47.220870,-1.552151,"Conseil départemental de Loire-Atlantique, 3, ..."
83,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,"21-29, avenue du Général de Gaulle Conseil gén...","Conseil général du Val de Marne, ETAI, AFASER",Financiers : ________________________ 10.000 €...,Afin de développer la participation de tous le...,"Accessibilité, Mobilité Intégration Lutte cont...",http://www.valdemarne.fr mailto:sophie.largeau...,Île-de-France,...,NaN,NaN,NaN,NaN,Conseil général du Val de Marne,"{'raw': {'place_id': '189432783', 'licence': '...","{'place_id': '189432783', 'licence': 'Data © O...",48.746489,2.409920,"Robert Desnos, Rue du Docteur Calmette, Port d..."
8,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: chargée de mission handicap et ...,"Préfecture, Direction régionale et département...",• Un chargé de mission analyse des besoins soc...,Afin d’améliorer la coordination et l’interpré...,"Gouvernance, partenariats institutionnels Diag...",http://www.bordeaux.fr mailto:c.beaufort-lance...,Aquitaine,...,NaN,NaN,NaN,NaN,Ville de Bordeaux,"{'raw': {'place_id': '82004620', 'licence': 'D...","{'place_id': '82004620', 'licence': 'Data © Op...",44.870552,-0.558876,"Base sous-marine de Bordeaux, Boulevard Alfred..."


,spider_name,link_src,link_data,fiche contact,date du projet,partenaires du projet,données économiques,résumé du projet,tags,website,...,titre du projet,image(s) du projet,structure porteuse,SIRET,statut juridique,récompenses,video,_full_address_,_location_,raw
19,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: 3 Quai Ceineray Tel: 02 40 99 1...,Le dispositif est entièrement internalisé. Il ...,Financiers : Quatre travailleurs sociaux du Dé...,Afin d’apporter une solution d’accueil adaptée...,Protection de l'enfance Intégration Nouvelles ...,mailto:laurent.gollandeau@loire-atlantique.fr ...,Pays de la Loire,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Conseil départemental de Loire-Atlantique,"{'raw': {'place_id': '241565046', 'licence': '...","{'place_id': '241565046', 'licence': 'Data © O..."
156,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Association la Source Lien internet: Mail: 3 r...,"En novembre 1994, la SCI des Amis de la Poulti...",___________________,>>Consulter la brochure des 20 ans de La Sourc...,Parentalité Lutte contre l'exclusion sociale P...,http://www.associationlasource.fr mailto:rober...,Haute-Normandie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Association la Source,"{'raw': {'place_id': '63721638', 'licence': 'D...","{'place_id': '63721638', 'licence': 'Data © Op..."
83,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,"21-29, avenue du Général de Gaulle Conseil gén...","Conseil général du Val de Marne, ETAI, AFASER",Financiers : ________________________ 10.000 €...,Afin de développer la participation de tous le...,"Accessibilité, Mobilité Intégration Lutte cont...",http://www.valdemarne.fr mailto:sophie.largeau...,Île-de-France,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Conseil général du Val de Marne,"{'raw': {'place_id': '189432783', 'licence': '...","{'place_id': '189432783', 'licence': 'Data © O..."
17,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Fabrique du quartier Danton – Ville du Havre L...,Sur la concertation menée sur le quartier Dant...,700 000 euros investis spécifiquement sur la c...,Afin de mettre les habitants au cœur du projet...,"Rénovation urbaine Développement urbain, Vie d...",mailto:benjamin.lebourg@lehavre.fr http://www....,Haute-Normandie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville du Havre,"{'raw': {'place_id': '22957963', 'licence': 'D...","{'place_id': '22957963', 'licence': 'Data © Op..."
22,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: 67076 Strasbourg Cedex Mail: Re...,Le projet est désormais porté et piloté par l’...,Une dizaine d’agrès de street workout. Financi...,Afin de favoriser l’insertion de jeunes du qua...,"Jeunesse Développement urbain, Vie des quartie...",mailto:lionel.boizot@strasbourg.eu http://www....,Alsace,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Eurométropole de Strasbourg,"{'raw': {'place_id': '81368751', 'licence': 'D...","{'place_id': '81368751', 'licence': 'Data © Op..."


In [ ]:
### cf : https://geopy.readthedocs.io/en/stable/#usage-with-pandas
geocoder_nom = Nominatim(user_agent="_solidata_app_")
geocoder_ban = BANFrance(user_agent="_solidata_app_")

### rate limiter
geocode_nom = RateLimiter(geocoder_nom.geocode, min_delay_seconds=dft_delay)
geocode_ban = RateLimiter(geocoder_ban.geocode, min_delay_seconds=dft_delay)

In [ ]:
df_01[location_col+"_nom"] = df_01[full_address_col].apply(partial(geocode_nom, language='en'))

In [ ]:
df_02[location_col+"_ban"] = df_02[full_address_col].progress_apply(geocode_ban)
# df_02[location_col+"_ban"] = df_02[full_address_col].apply(geocode_ban)

In [ ]:
df_02.sample(10)